<a href="https://colab.research.google.com/github/fabiansiegel/digitalkompetenzen/blob/main/musterloesung_datascience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialisierung

In [ ]:
# Packages installieren (ggf. beim ersten Lauf aktivieren, danach auskommentieren)
#!pip install pandas
#!pip install arrow
#!pip install folium
#!pip install matplotlib
#!pip install scipy
#!pip install seaborn

import pandas as pd
import datetime
import folium as fol
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import math
import arrow
from scipy.stats import chi2_contingency
import seaborn as sns


## Data Prep

### Patientenstammdaten laden

In [ ]:
base_url="https://raw.githubusercontent.com/miracum/c19ss2021/master/uebung-5/"
dateparse = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d')
patientsBielefeld=pd.read_csv(base_url+"rohdaten/bielefeld/csv/patients.csv", parse_dates=['BIRTHDATE'], date_parser=dateparse)
patientsBielefeld.head()

### Geovisualisierung der Patientenstammdaten

In [ ]:
# import folium as fol
# from folium.plugins import MarkerCluster


standortListeBielefeld=[] # für die Speicherung der Koordinaten der Patienten
for daten in range(patientsBielefeld.__len__()): # Die Daten werden mit Hilfe einer Schleife ausgelesen
    locationLat=patientsBielefeld["LAT"][daten]
    locationLon=patientsBielefeld["LON"][daten]
    standortListeBielefeld.append((locationLat, locationLon))

geomapBielefeld=fol.Map(location=[51.151509, 10.217850], zoom_start=6) 

#für die Clusteranzeige
marker_cluster = MarkerCluster().add_to(geomapBielefeld)
for point in range(0,patientsBielefeld.__len__()):
    fol.Marker(standortListeBielefeld[point]).add_to(marker_cluster)
geomapBielefeld

### Histogramm nach Geburtsjahr

In [ ]:
# import matplotlib.pyplot as plt

# Nur das Jahr der Gebursdaten wird betrachtet.
geburtsjahrlisteBielefeld=patientsBielefeld["BIRTHDATE"].dt.year.tolist()
# Die Geburtsdaten werden je nach Geschlecht in einer Liste abgelegt um sie getrennt voneinander darstellen zu können.
frauenlisteGeburtBielefeld=patientsBielefeld["BIRTHDATE"].loc[patientsBielefeld["GENDER"]=="F"].dt.year.tolist()
männerlisteGeburtBielefeld=patientsBielefeld["BIRTHDATE"].loc[patientsBielefeld["GENDER"]=="M"].dt.year.tolist()
diverslisteGeburtBielefeld=patientsBielefeld["BIRTHDATE"].loc[(patientsBielefeld["GENDER"]!="F") &(patientsBielefeld["GENDER"]!="M")].dt.year.tolist()



plt.hist(data=patientsBielefeld, x=frauenlisteGeburtBielefeld, bins=5, color="red", label="F")
plt.hist(data=patientsBielefeld, x=männerlisteGeburtBielefeld, bins=5, color="green", label="M")
# plt.hist(data=patientsBielefeld, x=diverslisteGeburtBielefeld, bins=5, color="white", label="D") # nicht befüllt
plt.legend()
plt.show()

### Alter berechnen & Histogramm nach Alter

In [ ]:
#import datetime

aktuellesDatum= datetime.date.today()

# wenn kein Todesdatum hinterlegt ist, wird das aktuelle Jahr eingetragen
patientsBielefeld["DEATHDATE"]=patientsBielefeld["DEATHDATE"].fillna(aktuellesDatum)
patientsBielefeld["DEATHDATE"]=pd.to_datetime(patientsBielefeld.DEATHDATE, format="%Y-%m-%d")
todeslistegesamtBielefeld=patientsBielefeld["DEATHDATE"].dt.year.tolist()

# DieTodesdaten den Geschlechtern zugeordnet
frauentodBielefeld=patientsBielefeld["DEATHDATE"].loc[patientsBielefeld["GENDER"]=="F"].dt.year.tolist()
männertodBielefeld=patientsBielefeld["DEATHDATE"].loc[patientsBielefeld["GENDER"]=="M"].dt.year.tolist()
diverstodBielefeld=patientsBielefeld["DEATHDATE"].loc[(patientsBielefeld["GENDER"]!="F") &(patientsBielefeld["GENDER"]!="M")].dt.year.tolist()



zaehlerGesamtAlter=0

gesamtAlterBielefeld=[]
# Mit Hilfe einer Schleife wird das Alter berechnet
# Dafür wird durch die Liste der jeweiligen Todesdaten iteriert und schließlich an dieser Stelle das Geburtsjahr genommen
# Mit Hilfe dieser Daten wird das Alter berechnet und dieses in eine neue Liste hinzugefügt
for todesjahrBielefeld in todeslistegesamtBielefeld:
    geburtsjahrBielefeld=geburtsjahrlisteBielefeld[zaehlerGesamtAlter]
    altergesamtBielefeld=todesjahrBielefeld-geburtsjahrBielefeld
    gesamtAlterBielefeld.append(altergesamtBielefeld)
    zaehlerGesamtAlter=zaehlerGesamtAlter+1

patientsBielefeld["AGE"]=gesamtAlterBielefeld
# Das Alter wird erneut je nach Geschlecht ermittelt um die Geschlechter unabhängig von einander darstellen zu können.
frauenalterListeBielefeld=patientsBielefeld["AGE"].loc[patientsBielefeld["GENDER"]=="F"].tolist()
männeralterListeBielefeld=patientsBielefeld["AGE"].loc[patientsBielefeld["GENDER"]=="M"].tolist()
diversalterListe=patientsBielefeld["AGE"].loc[(patientsBielefeld["GENDER"]!="F") &(patientsBielefeld["GENDER"]!="M")].tolist()

plt.hist(data=patientsBielefeld, x=frauenalterListeBielefeld, bins=5, color="red", label="F")
plt.hist(data=patientsBielefeld, x=männeralterListeBielefeld, bins=5, color="green", label="M")
plt.xlabel('Alter')
plt.ylabel('Anzahl')
plt.legend()
plt.show()

### Diagnosen laden

In [ ]:
diagnosenBielefeld=pd.read_csv(base_url+"rohdaten/bielefeld/csv/conditions.csv",parse_dates=["START"], date_parser=dateparse)
diagnosenBielefeld.head()

### Exploration der Diagnosen

In [ ]:
diagnosenBielefeld.groupby('DESCRIPTION').count().plot(kind="bar")


### Top-10 Diagnosen

In [ ]:
# Für die Top Diagnosen wird erst das Vorkommen gezählt und davon die ersten 10 genommen, also jene mit der größten Anzahl
diagnosenBielefeld["DESCRIPTION"].value_counts()[:10].plot(kind="bar")

## Auswertung der Daten

### Patient:innen mit Covid-19-Diagnosen auswählen

In [ ]:
diagnosenBielefeldcovid=pd.DataFrame(diagnosenBielefeld.PATIENT[diagnosenBielefeld.DESCRIPTION=="COVID-19"])
diagnosenBielefeldcovid.head()

### Ergänzung demographische Angaben

In [ ]:
# Die Spalte wird für den merge umbenannt
patientsBielefeld=patientsBielefeld.rename(columns={"Id":"PATIENT"})

# Daten werden zusammengeführt
bielefeldDiagnosenundPatienten=diagnosenBielefeldcovid.merge(patientsBielefeld, how="inner", on="PATIENT")
bielefeldDiagnosenundPatienten = bielefeldDiagnosenundPatienten[['PATIENT', 'BIRTHDATE', 'DEATHDATE', 'GENDER','AGE', 'CITY']]
bielefeldDiagnosenundPatienten

### Ableitung Vitalstatus aus Todesdatum

In [ ]:
#import math
#import arrow
vitalstatuslisteBielefeld=[]
aktuellesJahr= datetime.date.today().year


# Die Schleife geht erneut die Todesdaten durch.
# Wenn kein Datum existiert wird davon ausgegangen, dass der Patient lebt
for todesdatum in bielefeldDiagnosenundPatienten["DEATHDATE"]:
    todesdatum=arrow.get(todesdatum).year
    if todesdatum==aktuellesJahr:
        vitalstatus="ALIVE"
        vitalstatuslisteBielefeld.append(vitalstatus)
    else:
        vitalstatus="DEATH"
        vitalstatuslisteBielefeld.append(vitalstatus)

bielefeldDiagnosenundPatienten["VITALSTATUS"]=vitalstatuslisteBielefeld
bielefeldDiagnosenundPatienten.head()

### Boxplot Alter vs. Vitalstatus

In [ ]:
bielefeldDiagnosenundPatienten.boxplot(column=["AGE"],by=["VITALSTATUS"])

### Selektion der Begleitdiagnosen der Kohorte

In [ ]:
nebendiagnosenBielefeld=diagnosenBielefeld.merge(bielefeldDiagnosenundPatienten, on="PATIENT")
nebendiagnosenBielefeld=nebendiagnosenBielefeld[(nebendiagnosenBielefeld["DESCRIPTION"]!="COVID-19")&(nebendiagnosenBielefeld["DESCRIPTION"]!= "Suspected COVID-19")]
nebendiagnosenBielefeld

   ### Top-10 Begleitdiagnosen der Kohorte

In [ ]:
nebendiagnosenBielefeld["DESCRIPTION"].value_counts()[:10].plot(kind="bar")

### Reduktion auf Nebendiagnosen Adipositas, Erschöpfung & Atemnot

In [ ]:
reduzierteNebendiagnosenBielefeld=nebendiagnosenBielefeld[(nebendiagnosenBielefeld["DESCRIPTION"]=="Body mass index 30+ - obesity (finding)")|(nebendiagnosenBielefeld["DESCRIPTION"]=="Fatigue (finding)")|(nebendiagnosenBielefeld["DESCRIPTION"]=="Respiratory distress (finding)")]
reduzierteNebendiagnosenBielefeld=reduzierteNebendiagnosenBielefeld.loc[:,["PATIENT", "DESCRIPTION"]]
reduzierteNebendiagnosenBielefeld=reduzierteNebendiagnosenBielefeld.groupby('PATIENT').apply(lambda grp: grp.assign(DIAGNOSISPRESENT=1))

reduziertUmgeordnetBielefeld=pd.pivot(data=reduzierteNebendiagnosenBielefeld, columns="DESCRIPTION", values="DIAGNOSISPRESENT",index="PATIENT")
reduziertUmgenanntBielefeld=reduziertUmgeordnetBielefeld.rename(columns={"Body mass index 30+ - obesity (finding)":"OBESITY","Fatigue (finding)":"FATIGUE", "Respiratory distress (finding)":"RESPIRATORY_DISTRESS" })

reduziertNebendiagnosenundPatientenBielefeld=bielefeldDiagnosenundPatienten.merge(reduziertUmgenanntBielefeld, on="PATIENT")
reduziertNebendiagnosenundPatientenBielefeld

### Generierung lokal aggregierter Zwischenergebnisse für die Merkmale

In [ ]:
del reduziertNebendiagnosenundPatientenBielefeld["AGE"]
lokalAggregiertBielefeld=reduziertNebendiagnosenundPatientenBielefeld.groupby("VITALSTATUS",dropna=True).sum()
lokalAggregiertBielefeld

### Chi-Quadrat-Test zwischen Vitalstatus & Nebendiagnosen durchführen

In [ ]:
# from scipy.stats import chi2_contingency
# import seaborn as sns

chi2,p, df, expctd= chi2_contingency(lokalAggregiertBielefeld)
print("x-squared= ", chi2, "p-value= ",p, "df= ", df, "\n expected= ", expctd)

sns.heatmap(lokalAggregiertBielefeld, cmap ='RdYlGn', linewidths = 0.30)


## Mannheimer Daten laden & identisch aufbereiten

In [ ]:
#import pandas as pd
#import math
#import datetime
#import arrow
base_url="https://raw.githubusercontent.com/miracum/c19ss2021/master/uebung-5/"
dateparse = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d')

# Daten Mannheim laden
patientsMannheim=pd.read_csv(base_url+"rohdaten/mannheim/csv/patients.csv", parse_dates=['BIRTHDATE'], date_parser=dateparse)
diagnosenMannheim=pd.read_csv(base_url+"rohdaten/mannheim/csv/conditions.csv",parse_dates=["START"], date_parser=dateparse)

# Alter ergänzen
aktuellesDatum = datetime.date.today()

geburtsjahrlisteMannheim=patientsMannheim["BIRTHDATE"].dt.year.tolist()
patientsMannheim["DEATHDATE"]=patientsMannheim["DEATHDATE"].fillna(aktuellesDatum)
patientsMannheim["DEATHDATE"]=pd.to_datetime(patientsMannheim.DEATHDATE, format="%Y-%m-%d")
todeslistegesamtMannheim=patientsMannheim["DEATHDATE"].dt.year.tolist()



zaehlerGesamtAlter=0

gesamtAlterMannheim=[]
for todesjahrMannheim in todeslistegesamtMannheim:
    geburtsjahrMannheim=geburtsjahrlisteMannheim[zaehlerGesamtAlter]
    altergesamtMannheim=todesjahrMannheim-geburtsjahrMannheim
    gesamtAlterMannheim.append(altergesamtMannheim)
    zaehlerGesamtAlter=zaehlerGesamtAlter+1

patientsMannheim["AGE"]=gesamtAlterMannheim

# Patient:innen mit Covid
diagnosenMannheimcovid=pd.DataFrame(diagnosenMannheim.PATIENT[diagnosenMannheim.DESCRIPTION=="COVID-19"])

# Ergänzung demographische Daten
patientsMannheim=patientsMannheim.rename(columns={"Id":"PATIENT"})
mannheimDiagnosenundPatienten=diagnosenMannheimcovid.merge(patientsMannheim, how="inner", on="PATIENT")
mannheimDiagnosenundPatienten = mannheimDiagnosenundPatienten[['PATIENT', 'BIRTHDATE', 'DEATHDATE', 'GENDER','AGE', 'CITY']]
mannheimDiagnosenundPatienten

# Vitalstatus ergänzen

aktuellesJahr = datetime.date.today().year

vitalstatuslisteMannheim=[]

for todesdatum in mannheimDiagnosenundPatienten["DEATHDATE"]:
    todesdatum=arrow.get(todesdatum).year
    if todesdatum==aktuellesJahr:
            vitalstatus="ALIVE"
            vitalstatuslisteMannheim.append(vitalstatus)
    else:
        vitalstatus="DEATH"
        vitalstatuslisteMannheim.append(vitalstatus)

mannheimDiagnosenundPatienten["VITALSTATUS"]=vitalstatuslisteMannheim

# Begleitdiagnosen bestimmen
nebendiagnosenMannheim=diagnosenMannheim.merge(mannheimDiagnosenundPatienten, on="PATIENT")
nebendiagnosenMannheim=nebendiagnosenMannheim[(nebendiagnosenMannheim["DESCRIPTION"]!="COVID-19")&(nebendiagnosenMannheim["DESCRIPTION"]!= "Suspected COVID-19")]

# Zu untersuchende Nebendiagnosen (Adipositas, Erschöpfung und Atemnot) auslesen
reduzierteNebendiagnosenMannheim=nebendiagnosenMannheim[(nebendiagnosenMannheim["DESCRIPTION"]=="Body mass index 30+ - obesity (finding)")|(nebendiagnosenMannheim["DESCRIPTION"]=="Fatigue (finding)")|(nebendiagnosenMannheim["DESCRIPTION"]=="Respiratory distress (finding)")]
reduzierteNebendiagnosenMannheim=reduzierteNebendiagnosenMannheim.loc[:,["PATIENT", "DESCRIPTION"]]
reduzierteNebendiagnosenMannheim=reduzierteNebendiagnosenMannheim.groupby('PATIENT').apply(lambda grp: grp.assign(DIAGNOSISPRESENT=1))

reduziertUmgeordnetMannheim=pd.pivot(data=reduzierteNebendiagnosenMannheim, columns="DESCRIPTION", values="DIAGNOSISPRESENT",index="PATIENT")
reduziertUmgenanntMannheim=reduziertUmgeordnetMannheim.rename(columns={"Body mass index 30+ - obesity (finding)":"OBESITY","Fatigue (finding)":"FATIGUE", "Respiratory distress (finding)":"RESPIRATORY_DISTRESS" })

reduziertNebendiagnosenundPatientenMannheim=mannheimDiagnosenundPatienten.merge(reduziertUmgenanntMannheim, on="PATIENT")


### Generierung lokal aggregierter Zwischenergebnisse für die Merkmale Mannheim

In [ ]:
del reduziertNebendiagnosenundPatientenMannheim["AGE"]
lokalAggregiertMannheim=reduziertNebendiagnosenundPatientenMannheim.groupby("VITALSTATUS",dropna=True).sum()
lokalAggregiertMannheim

### Chi-Quadrat-Test für Mannheimer-Daten durchführen

In [ ]:
#from scipy.stats import chi2_contingency
#import seaborn as sns

chi2,p, df, expctd= chi2_contingency(lokalAggregiertMannheim)
print("x-squared= ", chi2, "p-value= ",p, "df= ", df, "\n expected= ", expctd)

sns.heatmap(lokalAggregiertMannheim, cmap ='RdYlGn', linewidths = 0.30)

## Leipziger Daten laden & identisch aufbereiten

In [ ]:
#import pandas as pd
#import math
#import datetime
#import arrow
base_url="https://raw.githubusercontent.com/miracum/c19ss2021/master/uebung-5/"
dateparse = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d')

# Daten Leipzig laden
patientsLeipzig=pd.read_csv(base_url+"rohdaten/leipzig/csv/patients.csv", parse_dates=['BIRTHDATE'], date_parser=dateparse)
diagnosenLeipzig=pd.read_csv(base_url+"rohdaten/leipzig/csv/conditions.csv",parse_dates=["START"], date_parser=dateparse)

# Alter ergänzen
aktuellesDatum = datetime.date.today()

geburtsjahrlisteLeipzig=patientsLeipzig["BIRTHDATE"].dt.year.tolist()
patientsLeipzig["DEATHDATE"]=patientsLeipzig["DEATHDATE"].fillna(aktuellesDatum)
patientsLeipzig["DEATHDATE"]=pd.to_datetime(patientsLeipzig.DEATHDATE, format="%Y-%m-%d")
todeslistegesamtLeipzig=patientsLeipzig["DEATHDATE"].dt.year.tolist()



zaehlerGesamtAlter=0

gesamtAlterLeipzig=[]
for todesjahrLeipzig in todeslistegesamtLeipzig:
    geburtsjahrLeipzig=geburtsjahrlisteLeipzig[zaehlerGesamtAlter]
    altergesamtLeipzig=todesjahrLeipzig-geburtsjahrLeipzig
    gesamtAlterLeipzig.append(altergesamtLeipzig)
    zaehlerGesamtAlter=zaehlerGesamtAlter+1

patientsLeipzig["AGE"]=gesamtAlterLeipzig
# Patient:innen mit Covid
diagnosenLeipzigcovid=pd.DataFrame(diagnosenLeipzig.PATIENT[diagnosenLeipzig.DESCRIPTION=="COVID-19"])

# Ergänzung demographische Daten
patientsLeipzig=patientsLeipzig.rename(columns={"Id":"PATIENT"})
leipzigDiagnosenundPatienten=diagnosenLeipzigcovid.merge(patientsLeipzig, how="inner", on="PATIENT")
leipzigDiagnosenundPatienten = leipzigDiagnosenundPatienten[['PATIENT', 'BIRTHDATE', 'DEATHDATE', 'GENDER','AGE', 'CITY']]
leipzigDiagnosenundPatienten

# Vitalstatus ergänzen
aktuellesJahr= datetime.date.today().year
vitalstatuslisteLeipzig=[]

for todesdatum in leipzigDiagnosenundPatienten["DEATHDATE"]:
    todesdatum=arrow.get(todesdatum).year
    if todesdatum==aktuellesJahr:
            vitalstatus="ALIVE"
            vitalstatuslisteLeipzig.append(vitalstatus)
    else:
        vitalstatus="DEATH"
        vitalstatuslisteLeipzig.append(vitalstatus)

leipzigDiagnosenundPatienten["VITALSTATUS"]=vitalstatuslisteLeipzig

# Begleitdiagnosen bestimmen
nebendiagnosenLeipzig=diagnosenLeipzig.merge(leipzigDiagnosenundPatienten, on="PATIENT")
nebendiagnosenLeipzig=nebendiagnosenLeipzig[(nebendiagnosenLeipzig["DESCRIPTION"]!="COVID-19")&(nebendiagnosenLeipzig["DESCRIPTION"]!= "Suspected COVID-19")]

# Zu untersuchende Nebendiagnosen (Adipositas, Erschöpfung und Atemnot) auslesen
reduzierteNebendiagnosenLeipzig=nebendiagnosenLeipzig[(nebendiagnosenLeipzig["DESCRIPTION"]=="Body mass index 30+ - obesity (finding)")|(nebendiagnosenLeipzig["DESCRIPTION"]=="Fatigue (finding)")|(nebendiagnosenLeipzig["DESCRIPTION"]=="Respiratory distress (finding)")]
reduzierteNebendiagnosenLeipzig=reduzierteNebendiagnosenLeipzig.loc[:,["PATIENT", "DESCRIPTION"]]
reduzierteNebendiagnosenLeipzig=reduzierteNebendiagnosenLeipzig.groupby('PATIENT').apply(lambda grp: grp.assign(DIAGNOSISPRESENT=1))

reduziertUmgeordnetLeipzig=pd.pivot(data=reduzierteNebendiagnosenLeipzig, columns="DESCRIPTION", values="DIAGNOSISPRESENT",index="PATIENT")
reduziertUmgenanntLeipzig=reduziertUmgeordnetLeipzig.rename(columns={"Body mass index 30+ - obesity (finding)":"OBESITY","Fatigue (finding)":"FATIGUE", "Respiratory distress (finding)":"RESPIRATORY_DISTRESS" })

reduziertNebendiagnosenundPatientenLeipzig=leipzigDiagnosenundPatienten.merge(reduziertUmgenanntLeipzig, on="PATIENT")


### Generierung lokal aggregierter Zwischenergebnisse für die Merkmale Leipzig

In [ ]:
del reduziertNebendiagnosenundPatientenLeipzig["AGE"]
lokalAggregiertLeipzig=reduziertNebendiagnosenundPatientenLeipzig.groupby("VITALSTATUS",dropna=True).sum()
lokalAggregiertLeipzig

### Chi-Quadrat-Test für Leipziger-Daten durchführen

In [ ]:
from scipy.stats import chi2_contingency
import seaborn as sns

chi2,p, df, expctd= chi2_contingency(lokalAggregiertLeipzig)
print("x-squared= ", chi2, "p-value= ",p, "df= ", df, "\n expected= ", expctd)

sns.heatmap(lokalAggregiertLeipzig, cmap ='RdYlGn', linewidths = 0.30)

## Lokale Zwischenergebnisse der 3 Standorte zusammenführen

In [ ]:
reduziertNebendiagnosenundPatientenAlleZusammen=reduziertNebendiagnosenundPatientenBielefeld.append([reduziertNebendiagnosenundPatientenMannheim,reduziertNebendiagnosenundPatientenLeipzig])

lokalAggregiertAlleZusammen=reduziertNebendiagnosenundPatientenAlleZusammen.groupby("VITALSTATUS",dropna=True).sum()
lokalAggregiertAlleZusammen

### Chi-Quadrat-Test über die zusammengeführten Daten durchführen


In [ ]:
from scipy.stats import chi2_contingency
import seaborn as sns

chi2,p, df, expctd= chi2_contingency(lokalAggregiertAlleZusammen)
print("x-squared= ", chi2, "p-value= ",p, "df= ", df, "\n expected= ", expctd)

sns.heatmap(lokalAggregiertAlleZusammen, cmap ='RdYlGn', linewidths = 0.30)